# Apertus Python Client – Hands-on Examples

This notebook demonstrates how to use the `apertus` client to call the Apertus (Public AI Gateway) inference API. It mirrors familiar OpenAI-style ergonomics and includes sync, streaming, and async flows.

In [6]:
# 1) Import and Auto-Reload
%load_ext autoreload
%autoreload 2

from apertus import Apertus, AsyncApertus, ApertusAPIError
from apertus.types import ChatMessage
import os, json, asyncio, time


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# 2) Local Install and Version Check
%pip install -e .

import sys, importlib.metadata as il
print("Python:", sys.version)
print("Apertus version:", il.version("apertus"))

Obtaining file:///home/svenp/Documents/apertus_chat
  Installing build dependencies ... -done
  Checking if build backend supports build_editable ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Getting requirements to build editable ... -done
  Preparing editable metadata (pyproject.toml) ... one
  Preparing editable metadata (pyproject.toml) ... -done
  Building editable for apertus (pyproject.toml) ... one
  Building editable for apertus (pyproject.toml) ... -done
  Created wheel for apertus: filename=apertus-0.1.0-0.editable-py3-none-any.whl size=3674 sha256=f824392118b3d00714cadab8a80d0b5ae3493e7e959275a0bd1710b1c76e810a
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdonf345/wheels/8a/49/1d/bcff3240cd9a3a7b6851df135623214d5b4300857a9d14f374
Successfully built apertus
done
  Created wheel for apertus: filename=apertus-0.1.0-0.editable-py3-none-any.whl size=3674 sha256=f824392118b3d00714cadab8a80d0b5ae3493e

In [ ]:
# 3) Quickstart: Core API Call — List Models and pick a model

client = Apertus(api_key="<YOUR_KEY>")
models = client.models.list()
print("Available models:", [m.id for m in models.data][:10])

model_id = models.data[0].id if models.data else None
print("Selected model:", model_id)

Available models: ['meta-llama/Llama-3.2-3B-Instruct', 'Cohere/Cohere-embed-multilingual-v3.0', 'Cohere/rerank-v3.5', 'swiss-ai/apertus-8b-instruct', 'swiss-ai/apertus-70b-instruct', 'openai/gpt-oss-120b', 'aisingapore/Gemma-SEA-LION-v3-9B-IT']
Selected model: meta-llama/Llama-3.2-3B-Instruct


In [10]:
# 4) Non-streaming Chat Completion with apertus-70b-instruct
assert model_id is not None, "No model available; ensure your API key has access."
resp = client.chat.completions.create(
    model="swiss-ai/apertus-70b-instruct",
    messages=[{"role": "user", "content": "Hello Apertus!"}],
    temperature=0.2,
    max_tokens=64,
)
print(resp.choices[0].message.content)

Hello! How can I assist you today? If you have any questions or need information, feel free to ask.


In [11]:
# 5) Streaming Chat Completion with apertus-70b-instruct
chunks = []
for ev in client.chat.completions.stream(
    model="swiss-ai/apertus-70b-instruct",
    messages=[{"role": "user", "content": "Stream a short greeting sentence."}],
    temperature=0.2,
    max_tokens=64,
):
    if ev.delta:
        print(ev.delta, end="", flush=True)
        chunks.append(ev.delta)
print()  # newline
streamed_text = "".join(chunks)

Hello!! I I'm'm Apert Apertusus,, your your AI AI assistant assistant.. How How can can I I assist assist you you today today??



In [ ]:
# 6) Async usage
async def async_demo():
    aclient = AsyncApertus(api_key="<YOUR_KEY>")
    models = await aclient.models.list()
    m = models.data[0].id if models.data else None
    assert m, "No model available"
    resp = await aclient.chat.completions.create(
        model=m,
        messages=[{"role": "user", "content": "Async hello!"}],
    )
    print(resp.choices[0].message.content)

await async_demo()

Async back at ya. How can I assist you today?


In [14]:
# 7) Error Handling and Logging
import logging
logging.basicConfig(level=logging.INFO)

try:
    bad = Apertus(api_key="invalid-key")
    bad.models.list()
except ApertusAPIError as e:
    print("Caught ApertusAPIError:", e.status_code, e.message)


INFO:httpx:HTTP Request: GET https://api.publicai.co/v1/models "HTTP/1.1 401 Unauthorized"


Caught ApertusAPIError: 401 {
  "type": "https://httpproblems.com/http-status/401",
  "title": "Unauthorized",
  "status": 401,
  "detail": "Authorization Failed",
  "instance": "/v1/models",
  "trace": {
    "timestamp": "2025-09-07T19:53:01.241Z",
    "requestId": "84cb014c-5cf7-47aa-8ebe-9f515c669a5c",
    "buildId": "52247c8c-ae84-448c-ab64-5d69a414580f",
    "rayId": "97b8bff54c42bc68"
  }
}


In [15]:
# 9) Performance: timeit
%timeit client.chat.completions.create(model=model_id, messages=[{"role": "user", "content": "ping"}], max_tokens=8)

INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/

859 ms ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# 10) Batch Processing with pandas + tqdm
import pandas as pd
from tqdm.auto import tqdm

prompts = pd.DataFrame({"prompt": [
    "Say hi in one word",
    "Say bye in one word",
    "Name a color",
]})

results = []
for row in tqdm(prompts.itertuples(index=False)):
    out = client.chat.completions.create(
        model="swiss-ai/apertus-70b-instruct",
        messages=[{"role": "user", "content": row.prompt}],
        max_tokens=16,
    )
    results.append(out.choices[0].message.content)

prompts["response"] = results
prompts.head()

/home/svenp/Documents/apertus_chat/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:00, ?it/s]INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
1it [00:01,  1.06s/it]INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
1it [00:01,  1.06s/it]INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
2it [00:01,  1.05it/s]INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
2it [00:01,  1.05it/s]INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
3it [00:02,  1.09it/s]
INFO:httpx:HTTP Request: POST https://api.publicai.co/v1/chat/completions "HTTP/1.1 200 OK"
3it [00:02,  1.09it/s]


,prompt,response
0,Say hi in one word,Hello!
1,Say bye in one word,Goodbye!
2,Name a color,"One color that could be named is ""cerulean"". \..."
